In [ ]:
using Pkg
if isfile("../Project.toml") && isfile("../Manifest.toml")
    Pkg.activate("..");
    ENV["PYTHON"] = "python3";
end

using Plots, JuliaProbo, Seaborn, StaticArrays
gr();

In [ ]:
mutable struct PolicyEvaluator
    pose_min::Vector{Float64}
    pose_max::Vector{Float64}
    reso::Vector{Float64}
    goal::Goal
    index_nums::Vector{Int64}
    indices::Vector{Tuple{Int64, Int64, Int64}}
    value_function_::AbstractArray{Float64, 3}
    final_state_flags_::AbstractArray{Float64, 3}
end

function set_final_state(
        reso::Vector{Float64},
        goal::Goal,
        pose_min::Vector{Float64},
        pose_max::Vector{Float64},
        index::Tuple{Int64, Int64, Int64}
    )
    x_min, y_min, ~ = pose_min .+ reso .* ([index...] .- 1)
    x_max, y_max, ~ = pose_max .+ reso .* ([index...] .- 1)
    
    corners = [[x_min, y_min], [x_max, y_min], [x_max, y_max], [x_min, y_max]]
    return convert(Float64, all( [inside(goal, corner) for corner in corners] ))
end

# constructor
function PolicyEvaluator(reso::Vector{Float64}, goal::Goal, lowerleft=[-4.0, -4.0], upperright=[4.0, 4.0])
    pose_min = vcat(lowerleft, [0.0])
    pose_max = vcat(upperright, [2pi])
    index_nums = [convert(Int64, round( (pose_max[i] - pose_min[i])/reso[i] )) for i in 1:3]
    indices = [ collect(Iterators.product(1:index_nums[1], 1:index_nums[2], 1:index_nums[3]))... ]
    v = @MArray zeros(index_nums[1], index_nums[2], index_nums[3])
    f = @MArray zeros(index_nums[1], index_nums[2], index_nums[3])
    for index in indices
        f[index...] = set_final_state(reso, goal, pose_min, pose_max, index)
        v[index...] = (f[index...] == 0.0f) ? goal.value : (-100.0)
    end
    
    return PolicyEvaluator(pose_min, pose_max, reso, goal, index_nums, indices, v, f)
end

In [ ]:
pe = PolicyEvaluator([1.0, 1.0, pi/4], Goal(-3.0, -3.0))